# Infromation Generator
fuente original: https://pgirish.github.io/spark-project/index.html

In [ ]:
#importemos la dependecias necesarias:
import socket, sys, requests
try:
    import requests_oauthlib
except:
    !pip install requests-oauthlib
    import requests_oauthlib
import json
import bleach
from bs4 import BeautifulSoup

In [ ]:
# a la aplicación se le ha dado el nombre de Spark Streaming & tweepy
# creemos las credenciales:
access_token = '#'
access_secret = '#'
consumer_key = '#' # api Key
consumer_secret = '#' # api secret

In [ ]:
# Creemos la variable de autenticación:
my_auth = requests_oauthlib.OAuth1(consumer_key, consumer_secret, access_token, access_secret)

In [ ]:
# creemos la funcion que obtendrá los twits:
def get_tweets():
    url = 'https://stream.twitter.com/1.1/statuses/filter.json'
    query_data = [('language', 'en'), ('locations', '-130,-20,100,50'),('track','#')]
    query_url = url + '?' + '&'.join([str(t[0]) + '=' + str(t[1]) for t in query_data])
    response = requests.get(query_url, auth=my_auth, stream=True) # obtiene los twits con los datos de autenticacion
    print(query_url, response)
    return response

In [ ]:
# ahora será necesaria una funcion que envie la infromación a spark
def send_tweets_to_spark(http_resp, tcp_connection):
    for line in http_resp.iter_lines():
        try:
            full_tweet = json.loads(line)
            tweet_text = full_tweet['text']# separa el texto del tweet del resto
            print("Tweet Text: " + tweet_text)
            #print ("------------------------------------------")
            tweet_screen_name = "SN:"+full_tweet['user']['screen_name']
            print("SCREEN NAME IS : " + tweet_screen_name)
            #print ("------------------------------------------")
            source = full_tweet['source']
            soup = BeautifulSoup(source)
            for anchor in soup.find_all('a'):         
               print("Tweet Source: " + anchor.text)        
            tweet_source = anchor.text
            source_device = tweet_source.replace(" ", "")
            device = "TS"+source_device.replace("Twitter", "") 
            print("SOURCE IS : " + device)
            #print ("------------------------------------------")
            tweet_country_code = "CC"+full_tweet['place']['country_code']
            print("COUNTRY CODE IS : " + tweet_country_code)#obtenemos el pais desde el que se ha creado
            print ("------------------------------------------")
            tcp_connection.send(
                tweet_text +' '+ tweet_country_code + ' '+ tweet_screen_name +' '+ device +'\n'
            )# envia la infromacion mediante protoclolo tcp
        except:
            continue


In [ ]:
# creemos ahora la configuración del generador de infromación:
TCP_IP = "localhost" # establecemos la IP
TCP_PORT = 9009 # establecemos el puerto de comunicación
conn = None
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.bind((TCP_IP, TCP_PORT))
s.listen(1)
print("Waiting for TCP connection...")
conn, addr = s.accept()
print("Connected... Starting getting tweets.")
resp = get_tweets() # obtenemos los tweets
send_tweets_to_spark(resp, conn) #los enviamos a spark mdiante el host y la IP seleccionados